In [1]:
# !unzip robohearts.zip
#%cd robohearts/

In [1]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo import MonteCarlo
from tqdm import tqdm_notebook

In [7]:
NUM_TESTS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100

run_train = True

playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

# Human vs Random
agent_list[0] = MonteCarlo(playersNameList[0], {'print_info' : False})
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [ ]:
# TRAIN THE MONTE CARLO AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
if run_train:
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
        observation = env.reset()
        history = []
        while True:
            #env.render()

            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)

            # update my agent
            if now_event == 'RoundEnd':
                agent_list[0].update_weights(history, -reward['MonteCarlo'])
                history = []
                weights = agent_list[0].weight_vec
                print(-reward['MonteCarlo'])
            if now_event == 'GameOver':
                  break
            if not IsBroadcast and observation['data']['playerName'] == 'MonteCarlo':
                history.append(observation)
            observation, reward, done, info = env.step(action)


-6
0
-3
-13
-20
-14
-4
0
-1
-22
-4
-4
-1
-3
-9
0
-5
-4
0
-24
-1
-5
-7
-5
-9
-2
-3
-3
-3
-7
-4
-6
-8
0
-17
-6
-9
-17
-5
-5
-20
-17
0
-20
-4
-18
-7
0
-4
-2
-4
-4
-17
-13
-3
-3
-13
-23
-8
0
0
0
-2
-20
0
-4
-19
-21
0
0
0
-18
-3
-8
-10
-3
-8
0
-3
-1
-1
-5
-1
-13
-6
-1
-14
-17
-13
-20
-8
-20
-1
0
0
-19
-16
-7
-20
-17
0
-4
0
-3
0
0
-9
-18
-4
-1
0
-18
0
-16
-7
-1
-5
-3
0
0
0
-4
-6
-21
-5
-19
-21
-1
0
-8
-4
-26
-1
0
-2
-22
0
0
-3
-22
-13
-3
-17
0
-19
-4
-13
-1
0
0
-19
-12
-13
0
0
-4
-13
-4
-1
-20
0
0
-21
-1
-4
-4
-7
0
0
-1
-4
-19
0
-1
-2
0
-23
0
-1
0
0
-15
-4
-13
-15
-17
-3
-7
-22
-13
-4
-11
-13
-3
-5
-1
0
0
-15
-7
-8
-3
-6
0
-14
-17
-14
0
0
-5
0
-2
-5
-19
-2
-9
-2
-2
-5
-2
-3
0
-5
-2
0
-3
0
-26
-14
-22
-5
-2
-6
0
-5
0
-1
-18
-1
0
0
-4
-20
-13
0
-5
0
-1
-19
-10
0
-1
-4
-16
-18
-4
-15
-5
-15
-3
0
-16
-5
-5
-16
-1
-19
0
-4
-13
-1
-8
-7
0
-2
-4
-2
-4
-20
0
-5
-18
-14
0
0
-5
-1
-1
0
-13
-5
0
-8
0
0
-8
0
0
-23
-18
-8
-4
-2
-15
-8
-1
-1
0
0
-12
-14
-6
-3
-1
-8
0
-13
-11
-2
-15
-1
-3
-4
0
-3
-13
-3
-1

-2
-1
-6
-2
-4
-4
-4
-4
-1
-8
-18
0
-4
-4
-4
0
-1
-6
-1
-5
-1
0
-1
-3
-1
-13
-17
-4
-2
0
-11
0
0
-16
0
0
0
-4
0
0
-17
-21
-1
-13
-13
-14
-6
-5
-7
-15
-13
-15
-7
0
-4
-7
-17
-5
-12
-2
-7
-13
-1
-3
-24
-1
0
0
-1
-16
-25
-4
-17
-24
0
0
-5
-13
-7
0
-8
-3
-3
-13
-5
-2
-21
0
-18
-22
0
-7
-1
-5
0
0
0
-5
-16
0
0
-4
-6
-6
0
-13
0
-6
-1
-5
-1
-6
-3
-12
0
-13
-16
0
-6
0
-2
-13
-6
-18
-8
-4
-4
0
0
-16
-14
-1
0
-5
0
-4
-9
-4
-4
0
0
-2
-4
-4
-5
-5
0
-2
-3
0
0
-17
0
0
-1
-26
-5
0
-4
-20
0
-5
-21
-9
-2
-22
-4
-17
-17
-7
-1
-5
-5
-3
-17
-2
-2
-4
0
-14
-5
-1
-4
-4
-6
0
-14
-1
-4
-4
0
0
-5
-5
-7
-7
0
0
-13
-9
-5
-19
-4
0
0
-10
-5
0
-16
-2
-17
-17
-20
0
-4
-5
-1
-1
0
-5
-1
-17
-5
-6
-14
-1
-8
-7
-16
-6
0
-1
-7
0
-13
-18
-4
0
-1
-3
-4
0
-13
-6
-7
-4
-18
-10
-3
-3
0
-16
-6
-7
-21
-1
-4
-7
-5
-7
0
-14
-4
-13
-5
-21
-4
-1
-1
-1
-1
0
0
-10
0
0
0
0
-13
-16
0
-20
-4
-13
-13
0
-7
-5
-1
-8
0
-8
-1
-17
-13
-15
-4
-5
-14
-16
0
-6
-6
-21
0
-2
0
-5
-14
-15
-5
0
-4
-3
-3
0
-15
-1
-3
-17
-1
-8
-20
-9
0
0
-8
-1
-12
-5
-1

-5
-3
-4
-5
-4
-3
-3
-7
-2
-2
-13
-5
-22
-5
0
-5
-1
-4
-1
-2
-13
-4
-3
-5
0
0
-13
0
-3
-4
-9
0
0
-4
-2
-4
-4
-4
-5
-19
0
0
0
-17
-14
-16
0
-21
0
-3
0
-6
0
-20
-8
-1
-7
-16
0
-14
-3
-17
-23
-2
-7
0
-1
0
-14
-13
-17
0
-7
-8
-1
-14
0
-5
-19
-8
-16
-4
-6
0
0
-4
-5
-4
-16
-3
-14
-1
-4
-20
0
-6
-5
-16
-17
-7
-2
0
0
-6
-5
-3
-8
-4
-6
-1
-13
0
-1
-17
-9
-3
-24
0
-4
-11
0
-3
-1
-17
-7
0
-18
-17
0
-13
-3
-18
-4
0
-16
-15
-1
-14
-19
-5
-14
0
-15
-1
-1
-3
-4
0
-4
-9
0
-8
-3
-2
-9
-14
-4
-1
0
-20
-1
-14
-1
-3
0
-10
0
-5
-13
0
-8
-18
-4
-19
-4
-20
0
-1
-7
0
-8
0
0
-5
0
-13
-5
-4
-4
-7
-17
-2
0
-2
-13
-18
-13
-4
-5
-1
-4
-1
-2
0
-18
0
-3
-1
-6
0
-2
-1
-12
0
-1
-5
-19
0
-1
-18
0
-12
-14
-4
0
-18
-14
-1
-1
0
0
-16
-4
-18
0
-24
-1
-4
0
-3
-1
-6
-26
-5
0
-3
-4
-17
-7
-8
-11
-1
-3
-17
0
-8
-5
-18
0
-5
0
-1
0
-4
-5
0
-2
0
0
0
-14
-14
-10
-6
0
0
-1
0
-13
-5
-16
0
-2
-5
-5
0
0
0
0
-18
-5
-1
0
0
-20
-21
-21
-5
0
-9
-5
0
-21
-17
0
-4
-3
-7
-12
-1
-16
0
0
-1
-9
-3
-2
-1
-1
-5
-10
0
-17
-15
-2
-7
-2
-1
-3
-13
-8

In [5]:
# Uncomment this line for pretrained weights
weights = [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117,
-2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411,
-0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984,
-0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125,
-0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258,
0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836,
0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969,
-2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846,
-0.41568405, 0.2782292, -1.23761129, -1.80559854]

# EVALUATE THE MONTE CARLO AGENT
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MonteCarlo(playersNameList[0], params={'weight_vec' : weights})
mc_wins = [0] * NUM_TESTS
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won
        
pool = multiprocessing.Pool(processes=NUM_TESTS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)
pool.close()
pool.join()











[295, 299, 311, 277, 305, 298, 301, 326, 297, 298]


In [7]:
# EVALUATE THE RANDOM AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
playersNameList[0] = 'Randman'
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = [0] * NUM_TESTS
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won
        
pool = multiprocessing.Pool(processes=NUM_TESTS)
rand_wins = pool.map(run_test, rand_wins)
print(rand_wins)
pool.close()
pool.join()











[251, 255, 243, 235, 238, 255, 269, 254, 241, 270]


In [8]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")
print(f"\n\nThe Monte Carlo weights are: {str(weights)}")

Monte Carlo won 300.7 times on average :: [295, 299, 311, 277, 305, 298, 301, 326, 297, 298]
Random won 251.1 times on average :: [251, 255, 243, 235, 238, 255, 269, 254, 241, 270]


The Monte Carlo weights are: [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117, -2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411, -0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984, -0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125, -0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258, 0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836, 0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969, -2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846, -0.41568405, 0.2782292, -1.23761129, -1.80559854]
